In [21]:
import csv
TRANSFERS_FILE = '../data/daohack/transfer_events.csv'
CREATED_FILE = '../data/daohack/create_events.csv'

In [22]:
def read_csv(filename):
    rows = []
    with open(filename) as f:
        r = csv.reader(f)
        i = 0
        for row in r:
            if i == 0:
                i += 1
                continue
            rows.append(row)
    return rows

In [23]:
def format_addr(a):
    return '0x' + a[2:]

def row_to_transfer(row):
    return {
        'type': 'transfer',
        'from': format_addr(row[0]),
        'to': format_addr(row[1]),
        'amount': int(row[2]),
        'block': int(row[7]),
        'txIdx': int(row[5])
    }

def row_to_create(row):
    return {
        'type': 'create',
        'to': format_addr(row[0]),
        'amount': int(row[1]),
        'block': int(row[6]),
        'txIdx': int(row[4])
    }

In [24]:
transfers = [row_to_transfer(row) for row in read_csv(TRANSFERS_FILE)]
creates = [row_to_create(row) for row in read_csv(CREATED_FILE)]

In [25]:
creates[0]

{'type': 'create',
 'to': '0xfbc1ab9a735012cf8aa4f8738c5811cdbf39a0c5',
 'amount': 1000000000000000000,
 'block': 1432254,
 'txIdx': 2}

In [26]:
transfers[0]

{'type': 'transfer',
 'from': '0x3f6519dc382a31f8d12f8d161b8151b40c457347',
 'to': '0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98',
 'amount': 10051094920000000000,
 'block': 1599237,
 'txIdx': 37}

In [27]:
from collections import defaultdict

# merge by block/evtIdx. then simulate in order
sorted_logs = sorted(creates + transfers, 
                     key=lambda l: (l['block'], l['txIdx']))

# simulate all events
balances = defaultdict(int)
for log in sorted_logs:
    if log['type'] == 'create':
        balances[log['to']] += log['amount']
    
    elif log['type'] == 'transfer':
        amount = log['amount']
        to = log['to']
        frm = log['from']
        if balances[frm] < amount:
            print('not enough balance for transfer! exiting')
            break
        
        balances[to] += amount
        balances[frm] -= amount
    
    else:
        print('untyped log!')

In [28]:
# filter down to balances with >0 tokens
positive_balances = {k: v for k, v in balances.items() if v > 0}

[len(balances), len(positive_balances)]

[33822, 23867]

In [32]:
import csv

# write to file
owners = list(positive_balances.keys())

with open('../data/daohack/owner_addresses.csv', 'w') as f:
    w = csv.writer(f)
    for owner in owners:
        w.writerow([owner])